In [1]:
import gym
import numpy as np

#settings
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 25000
SHOW_EVERY = 2000


env = gym.make("MountainCar-v0")

discreate_os_size = [20,20]
discreate_os_win_size = (env.observation_space.high - env.observation_space.low)/discreate_os_size

q_table = np.random.uniform(low = -2, high = 0, size = (discreate_os_size)+[env.action_space.n])

def get_discreate_state(state):
    ostate = (state - env.observation_space.low)/discreate_os_win_size
    return tuple(ostate.astype(np.int))


for episode in range (EPISODES):
    if episode%SHOW_EVERY == 0:
        render = True
    else:
        render = False
    discreate_state = get_discreate_state(env.reset())
    done = False
    while not done:
        action = np.argmax(q_table[discreate_state])
        new_state, reward, done, _ = env.step(action)
        new_discreate_state = get_discreate_state(new_state)
        if render:
            env.render()

        if not done:
            max_future_q = np.max(q_table[new_discreate_state])
            curr_q = q_table[discreate_state + (action,)]
            new_q = (1-LEARNING_RATE) * curr_q + LEARNING_RATE *(reward + DISCOUNT *max_future_q)
            q_table[discreate_state+ (action, )] = new_q

        elif new_state[0] >= env.goal_position:
            q_table[discreate_state + (action, )] = 0

        discreate_state = new_discreate_state
env.close()    

KeyboardInterrupt: 

In [12]:
'''

env.action_space.n will give the no of possible moves (3 means 0,1,2)

'''

'\n\nenv.action_space.n will give the no of possible moves (3 means 0,1,2)\n\n'

In [9]:
import numpy as np
import gym
import matplotlib.pyplot as plt

# Import and initialize Mountain Car Environment
env = gym.make('MountainCar-v0')
env.reset()

# Define Q-learning function
def QLearning(env, learning, discount, epsilon, min_eps, episodes):
    # Determine size of discretized state space
    num_states = (env.observation_space.high - env.observation_space.low)*\
                    np.array([10, 100])
    num_states = np.round(num_states, 0).astype(int) + 1
    
    # Initialize Q table
    Q = np.random.uniform(low = -1, high = 1, 
                          size = (num_states[0], num_states[1], 
                                  env.action_space.n))
    
    # Initialize variables to track rewards
    reward_list = []
    ave_reward_list = []
    
    # Calculate episodic reduction in epsilon
    reduction = (epsilon - min_eps)/episodes
    
    # Run Q learning algorithm
    for i in range(episodes):
        # Initialize parameters
        done = False
        tot_reward, reward = 0,0
        state = env.reset()
        
        # Discretize state
        state_adj = (state - env.observation_space.low)*np.array([10, 100])
        state_adj = np.round(state_adj, 0).astype(int)
    
        while done != True:   
            # Render environment for last five episodes
            env.render()
                
            # Determine next action - epsilon greedy strategy
            if np.random.random() < 1 - epsilon:
                action = np.argmax(Q[state_adj[0], state_adj[1]]) 
            else:
                action = np.random.randint(0, env.action_space.n)
                
            # Get next state and reward
            state2, reward, done, info = env.step(action) 
            
            # Discretize state2
            state2_adj = (state2 - env.observation_space.low)*np.array([10, 100])
            state2_adj = np.round(state2_adj, 0).astype(int)
            
            #Allow for terminal states
            if done and state2[0] >= 0.5:
                Q[state_adj[0], state_adj[1], action] = reward
                
            # Adjust Q value for current state
            else:
                delta = learning*(reward + 
                                 discount*np.max(Q[state2_adj[0], 
                                                   state2_adj[1]]) - 
                                 Q[state_adj[0], state_adj[1],action])
                Q[state_adj[0], state_adj[1],action] += delta
                                     
            # Update variables
            tot_reward += reward
            state_adj = state2_adj
        
        # Decay epsilon
        if epsilon > min_eps:
            epsilon -= reduction
        
        # Track rewards
        reward_list.append(tot_reward)
        
        if (i+1) % 100 == 0:
            ave_reward = np.mean(reward_list)
            ave_reward_list.append(ave_reward)
            reward_list = []
            
        if (i+1) % 100 == 0:    
            print('Episode {} Average Reward: {}'.format(i+1, ave_reward))
            
    env.close()
    
    return ave_reward_list

# Run Q-learning algorithm
rewards = QLearning(env, 0.2, 0.9, 0.8, 0, 5000)

# Plot Rewards
plt.plot(100*(np.arange(len(rewards)) + 1), rewards)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward vs Episodes')
plt.savefig('rewards.jpg')     
plt.close() 

NotImplementedError: abstract